# Step 2 - Find the duplicates!
We have a file with 110.000.000 password of 20 characters. We have to define a hash function that associates a value to each string. The goal is to check whether there are some duplicate strings, in 2 cases:

- order is not important "AABA" = "AAAB"
- order is important "AABA" != "AAAB"

### Import the Libraries

In [1]:
import numpy as np
from collections import defaultdict
from bitarray import bitarray
import pickle

# Find the duplicates not considering the order of the password's charachters

## First Part

In [2]:
######## LOAD 1 #########
# Load all the data 
with open("passwords2.txt") as file:
    content = file.readlines()

### How we built the hash function
Our basic idea to give a hash code, possibly large and unique, to a password is use the [Fundamental Theorem of Arithmetic](https://en.wikipedia.org/wiki/Fundamental_theorem_of_arithmetic).

Shortly it says that each positive integer can be factorized in prime numbers in unique way. This will guarantee us the unicity of hash code for each password, less than the order. The importance of the unicity is explained in the second part.

In order to do this we have to associate a prime number to each character so we search all the possible character in the txt file like this.

In [3]:
# initialize an empty set
final_set = set()
# iterating over all the passwords...
for psw in content:
    # remove final "\n"
    psw = psw[:-1]
    # ... we store all the charachter using the union function in the set library
    final_set = final_set.union( set(psw) )

In [4]:
len(final_set)

84

We have 84 different character!

After this we built a function that check if a number is prime or not. Because now we need the first 84 prime number to associate them to the 84 characters stored in "final_set"

In [5]:
# this function check if the number in input is prime
def is_prime(n):
    # if n is equal to 2 is prime
    if n == 2:
        return True
    # we search a divider in all the numbers among 2 and the squared root of the number to check
    for i in range(2,int(np.sqrt(n))+1):
        # if we find a divider return False because the number isn't prime
        if n % i == 0:
            return False
    # at the end if we don't find any divider n is prime so return True
    return True

So in the first part of the next chunk we find the first 84 prime numbers and we store them in "list_of_prime".

Then we create the association: prime number with character, through the dictionary map_char_prime. It has the following shape:

map_char_prime = { "a" : 2, "$" : 3, "M" : 5,  ...  , "+" : 433 }

Then we save it in order not to repeat these operations each time we run the notebook. We choose json format to store the dictionary.

In [6]:
num_char = len(final_set) # how many prime number we need

# find the first "num_char" prime number
i = 2
list_of_prime = []
while len(list_of_prime) < num_char:
    if is_prime(i):
        list_of_prime.append(i)
        i += 1
        continue
    else:
        i += 1
        continue

# build a map from character to a prime number
map_char_prime = defaultdict(int)
for i in range(num_char):
    map_char_prime[list(final_set)[i]] = list_of_prime[i]

Now we have to associate a huge number to each password. Moreover, for the first task, we want that passwords that has the same character in different order, will have the same hash number. To do this we take advantage of commutativity of the product. 

Simply the hash code is the product among the prime numbers associated to the character of the password. In this way:
- Two equal password, NOT considering the order, have the same hash number
- And we can't have 2 different passwords that have the same hash number for the unicity of the factorization in prime numbers

The next function take in input the string password and return the hash number described.

In [7]:
def map_password_to_prime( password ):
    # create a list with all the character in the string casting it
    list_of_char = list(password)
    # this number will contain the hash code
    final_num = 1
    # iterate over the charachters
    for character in list_of_char:
        # multiplicate the current hash code with the prime number associated to the new charachter
        final_num *= map_char_prime[character]
    return(final_num)

Now we want to give an hash code to all the passwords and do the hash map. We can't use variable like: list, dataframe or dictionary to store the hash codes, because they will have a size too huge. Then we think that the cheapest choice is to use a binary array (using bitarray library). The map is done making the modulo operation, over the size of this array, of the hash code. In this way we find the position of the binary array. 

Then if the position is 0 we swap it into 1, but if it's already setted to 1 we have a collision.
The collision could be due to a true duplicate or a false positive. However we count all the collision in the variable "duplicate_count" and we save the collision position in a set for the next part where we will disinuge between True and False duplicates.

In [8]:
# initialize a binary array with 10^11 space with all zeros
a = bitarray(100000000003)
a.setall(0)
n_a = len(a)
duplicate_pos_set = set()

# initialize the counter for the duplicates
duplicate_count = 0
# search in all the password
for psw in content:
    # we decide the position in the bit array doing the hash code modulo the length of the bit array
    bit_position = map_password_to_prime( psw[:-1] ) % n_a
    # if an element is already in that position increase the duplicates counter
    if a[bit_position] == True:
        duplicate_count += 1
        # we store the position with the possible duplicates for the second part of the analysis
        duplicate_pos_set.add(bit_position)
    # if the element in that position is zero turn it to one ( or True )
    else:
        a[bit_position] = True

How many possible duplicates we have?

In [9]:
duplicate_count

10050010

With different attempts we got these results using different number for the modulo:

- with 110000017 we have 44318083 possible duplicates
- with 500000003 we have 19366303 possible duplicates
- with 1000000007 we have 14837676 possible duplicates
- with 9000000000 we have 10930362 possible duplicates
- with 100000000003 we have 10050010 possible duplicates

We know by the given result that there are 50010 false positives. Our hash map is really good it makes a mistake (false duplicates over possible duplicates) of 0.005%. 

But what about if we don't know the real number of duplicates? How we can understand it? Next we consider this problem. The thing that allows us to do it is that now we have to check only this 10 million of possible duplicates and not all the 110 million!

N.B. we use 100000000003 as module because we need a large prime number. According to our [research](https://shanghaiseagull.com/index.php/2017/09/11/why-do-hash-functions-use-prime-numbers/) is better to use a prime number to increase the sparsity of the bitarray

In [10]:
del(a)

Store in external file the set with all the positions of the possible duplicates in the bitarray. In this way we can avoid repeating the run of all the cells of the first part.

In [11]:
with open("position_set_1.txt", "wb") as fp:
    pickle.dump(duplicate_pos_set, fp)

## Second part

In [ ]:
########### LOAD 2 #############
with open("position_set_1.txt", "rb") as fp:
    duplicate_pos_set = pickle.load(fp)

Now we do the same thing done before, but when we find a bit position that is in the "duplicate_pos_set" (all the possible duplicate positions in the bitarray) we save the hash in a set. But if already exists another equal hash code in this set this is a true duplicate then we increase the counter "real_duplicate". This thing is possible only because we use a unique hash code for each password unique to less than the order!! Now is explained the choice of the use of the Fundamental Theorem of Arithmetic.

In [12]:
a = bitarray(100000000003)
a.setall(0)
n_a = len(a)

# initialize the counter for the real duplicates
real_duplicate = 0
# and the set will contain all the hash codes 
duplicate_hash_set = set()

# search in all the password
for psw in content:
    # we decide the position in the bit array doing the hash code modulo the length of the bit array
    hash_code = map_password_to_prime( psw[:-1] )
    bit_position = hash_code % n_a
    # if the bit position is one of those to be checked...
    if bit_position in duplicate_pos_set:
        # if the hash code is already saved increase the counter of the real duplicates
        if hash_code in duplicate_hash_set:
            real_duplicate += 1
        # otherwise save the hash code in its set
        else:
            duplicate_hash_set.add(hash_code)

At the end this is the real number of duplicate passwords

In [13]:
real_duplicate

10000000

Free the cache for the next resarch

In [14]:
del(a)
del(duplicate_hash_set)
del(duplicate_pos_set)

# Find the duplicates considering the order of the password's charachters

## First part
The first step is define an hash function that consider the position of the charachter. We know that the possible charachters are in total 84 and we give a number of to digits to each of them. In example:

"a" -> 10 , "b" -> 11, "c" -> 12, ... , "$" ->94

In [15]:
# define new dictionray
dict_char_position = defaultdict(int)
# start from 10 because we want that each charachter has to digits
i = 10
for charac in list(map_char_prime.keys()):
    # create the keys with the character and its values with the next two digits number
    dict_char_position[charac] = i
    i += 1

Then we give as hash code corresponding to a string, a number that is the substitute of the charachter in invers order of appearing. For instance:

"abc" -> 121110

because "a" is 10, "b" is 11 and "c" is 13

In [16]:
def map_hash_position( stringa ):
    # this number will contain the final hash code
    final_num = 0
    # iterate over the charachters in the string and their psition
    for pos, el in enumerate(list(stringa)):
        # we add to final number the current number of the new charachter
        final_num += dict_char_position[el] * ((100)**pos)
    return final_num

Then we do the same thing done before: do the module operation of the hash code with a large number and use the bit array for the hash map. Moreover we save all the position of the possible duplicates in a set for the second part, where we distingue the true and the false duplicates

In [17]:
# initialize the bitarray for the hash map with all zeros
a_2 = bitarray(100000000003)
a_2.setall(0)
n_a_2 = len(a_2)
# initialize the set will contain the positions of the possible duplicates
duplicate_pos_set_2 = set()
# initialize a counter for all the possible duplicates when we have a collision
duplicate_count_2 = 0
for psw in content:
    # compute the hash code and find its position in the array
    hash_code = map_hash_position( psw[:-1] )
    bit_position = hash_code % n_a_2
    # if we already have a 1 in this position increase the possible duplicate counter
    # and then add this position to the set of the possible duplicates
    if a_2[bit_position] == True:
        duplicate_count_2 += 1
        duplicate_pos_set_2.add(bit_position)
    # otherwise swap the 0 into a 1
    else:
        a_2[bit_position] = True

How many possible duplicates we have?

In [18]:
duplicate_count_2

5055060

Store the set with all the positions of the possible duplicates in the bitarray. In this way we can avoid repeating the run of all the cells of the first part

In [19]:
with open("position_set_2.txt", "wb") as fp:
    pickle.dump(duplicate_pos_set_2, fp)

Free the cache for the second part

In [20]:
del(a_2)

## Second part

In [ ]:
########### LOAD 3 #############
with open("position_set_2.txt", "rb") as fp:
    duplicate_pos_set_2 = pickle.load(fp)

Now the number of possible duplicates (true and false positive) is really decrease. We start with 110*10^6 passwords to check and after using the hash map we have only 5055060 elements to verify. We do the same thing done in the previous part using the unicty of the new hash code defined.

In [21]:
a_2 = bitarray(100000000003)
a_2.setall(0)
n_a_2 = len(a_2)

# initialize the counter for the real duplicates
real_duplicate_2 = 0
# and the set will contain all the hash codes 
duplicate_hash_set_2 = set()

# search in all the password
for psw in content:
    # we decide the position in the bit array doing the hash code modulo the length of the bit array
    hash_code = map_hash_position( psw[:-1] )
    bit_position = hash_code % n_a_2
    # if the bit position is one of those to be checked...
    if bit_position in duplicate_pos_set_2:
        # if the hash code is already saved increase the counter of the real duplicates
        if hash_code in duplicate_hash_set_2:
            real_duplicate_2 += 1
        # otherwise save the hash code in its set
        else:
            duplicate_hash_set_2.add(hash_code)

In [22]:
real_duplicate_2

5000000

The true duplicates number is 5000000 and we have 55060 false positive. With this number we can evaluate the goodness of the hash map that's really good beacuse it makes a mistake (false duplicates over possible duplicates) of 0.01%.

In [23]:
del(a_2)
del(duplicate_hash_set_2)
del(duplicate_pos_set_2)